## This notebook is an example of how to pipeline two models. 
This notebook is an example how to use DeGirum PySDK to do AI inference of a graphical file using two AI models: person detection and pose detection. The person detection model is run on the image and the results are then processed by the pose detection model, one person bbox at a time.
Combined result is then displayed.
This script uses PIL as image processing backend.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, mytools

cloud_token = mytools.get_token() # get cloud API access token from env.ini file
cloud_zoo_url = mytools.get_cloud_zoo_url() # get cloud zoo URL from env.ini file

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

# 1. Inference on the DeGirum Cloud Platform
zoo = dg.connect(dg.CLOUD, cloud_zoo_url, cloud_token)

# 2. Inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN
# zoo = dg.connect(mytools.get_ai_server_hostname(), cloud_zoo_url, cloud_token)

# 3. Inference on DeGirum ORCA accelerator installed on your computer
# zoo = dg.connect(dg.LOCAL, cloud_zoo_url, cloud_token)

#### The rest of the cells below should run without any modifications

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
people_det_model = zoo.load_model("yolo_v5s_person_det--512x512_quant_n2x_orca_1")
pose_model = zoo.load_model("mobilenet_v1_posenet_coco_keypoints--353x481_quant_n2x_orca_1")

# adjust some model properties
pose_model.overlay_show_labels = False
pose_model.overlay_line_width = 1
pose_model.overlay_alpha = 1
pose_model._model_parameters.PoseThreshold = 0.5

In [ ]:
# detect persons
people = people_det_model('./images/ThreePersons.jpg')

# for each detected person detect poses
with pose_model: # performance optimization to keep connection to pose_model open
    for person in people.results:
        person_box = people.image.crop(person['bbox'])
        poses = pose_model(person_box)
        display(poses.image_overlay)

In [ ]:
# alternatively, detect all poses
all_poses = pose_model('./images/ThreePersons.jpg')
display(all_poses.image_overlay)